In [5]:
import cv2
import sys
 
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
 
if __name__ == '__main__' :

    # Using the HOG Descriptor to dectect people in the video
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
    
    # Set up tracker.
    tracker_types = ['MIL','KCF']
    tracker_type = tracker_types[1]

    if int(minor_ver) < 3:
        tracker_1 = cv2.Tracker_create(tracker_type)
        tracker_2 = cv2.Tracker_create(tracker_type)
        tracker_3 = cv2.Tracker_create(tracker_type)
        tracker_4 = cv2.Tracker_create(tracker_type)
    else:
        if  tracker_type == 'MIL':
            tracker_1 = cv2.TrackerMIL_create()
            tracker_2 = cv2.TrackerMIL_create()
            tracker_3 = cv2.TrackerMIL_create()
            tracker_4 = cv2.TrackerMIL_create()

        elif tracker_type == 'KCF':
             tracker_1 = cv2.TrackerKCF_create()
             tracker_2 = cv2.TrackerKCF_create()
             tracker_3 = cv2.TrackerKCF_create()
             tracker_4 = cv2.TrackerKCF_create()

 
    # Read video
    video = cv2.VideoCapture("../Actividades/Semana 4/Recursos/people.mp4")

 
    # Exit if video not opened.
    if not video.isOpened():
        print("Could not open video")
        sys.exit()
 
    # Read first frame.
    ok, frame = video.read()
    if not ok:
        print ('Cannot read video file')
        sys.exit()
    
    # Using a greyscale picture, also for faster detection
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY) 

    # It returns the bounding boxes for the detected people
    (boxes,_) = hog.detectMultiScale(gray, winStride=(5,5))
    
    # Define an initial bounding box for each detection
    bbox_1 = boxes[0]
    bbox_2 = boxes[1]
    bbox_3 = boxes[2]
    bbox_4 = boxes[3]
 
 
    # Initialize tracker with first frame and bounding box
    ok = tracker_1.init(frame, bbox_1)
    ok = tracker_2.init(frame, bbox_2)
    ok = tracker_3.init(frame, bbox_3)
    ok = tracker_4.init(frame, bbox_4)
 
    while True:
        # Read a new frame
        ok, frame = video.read()
        if not ok:
            break
         
        # Start timer
        timer = cv2.getTickCount()
 
        # Update tracker
        ok, bbox_1 = tracker_1.update(frame)
        ok, bbox_2 = tracker_2.update(frame)
        ok, bbox_3 = tracker_3.update(frame)
        ok, bbox_4 = tracker_4.update(frame)
 
        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)
 
        # Draw bounding box
        if ok:
            # bbox_1
            point_1 = (int(bbox_1[0]), int(bbox_1[1]))
            point_2 = (int(bbox_1[0] + bbox_1[2]), int(bbox_1[1] + bbox_1[3]))
            cv2.rectangle(frame, point_1, point_2, (255,0,0), 2, 1)
            cv2.putText(frame, "Person 1", point_1, cv2.FONT_HERSHEY_SIMPLEX, 0.60,(255,255,255), 2)  
            
            # bbox_2
            point_3 = (int(bbox_2[0]), int(bbox_2[1]))
            point_4 = (int(bbox_2[0] + bbox_2[2]), int(bbox_2[1] + bbox_2[3]))
            cv2.rectangle(frame, point_3, point_4, (255,0,0), 2, 1)
            cv2.putText(frame, "Person 2", point_3, cv2.FONT_HERSHEY_SIMPLEX, 0.60,(255,255,0), 2)

            # bbox_3
            point_5 = (int(bbox_3[0]), int(bbox_3[1]))
            point_6 = (int(bbox_3[0] + bbox_3[2]), int(bbox_3[1] + bbox_3[3]))
            cv2.rectangle(frame, point_5, point_6, (255,0,0), 2, 1)
            cv2.putText(frame, "Person 3", point_5, cv2.FONT_HERSHEY_SIMPLEX, 0.60,(0,255,255), 2)  
            
            # bbox_4
            point_7 = (int(bbox_4[0]), int(bbox_4[1]))
            point_8 = (int(bbox_4[0] + bbox_4[2]), int(bbox_4[1] + bbox_4[3]))
            cv2.rectangle(frame, point_7, point_8, (255,0,0), 2, 1)
            cv2.putText(frame, "Person 4", point_7, cv2.FONT_HERSHEY_SIMPLEX, 0.60,(0,255,0), 2)    
        else :
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
 
        # Display tracker type on frame
        cv2.putText(frame, tracker_type + " Tracker", (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255),2)
     
        # Display FPS on frame
        cv2.putText(frame, "FPS : " + str(int(fps)), (10,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255), 2)
        
        # Show the number of people
        cv2.putText(frame, f"Number of people: {len(boxes)}", (200,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255), 2)

        # Display result
        cv2.imshow("Tracking", frame)
 
        # Exit if ESC pressed
        if cv2.waitKey(1) & 0xFF == ord('q'): # if press SPACE bar
            break
    video.release()
    cv2.destroyAllWindows()